In [3]:
# imports 
import pandas as pd
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
import json

In [2]:
df = pd.read_csv("palmerpenguins.csv")
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [31]:
app = Dash(__name__)

# generate scatter plot for selections 
scatter1 = px.scatter(df, x = "bill_length_mm", y = "bill_depth_mm", color = "species")
scatter1.update_layout(dragmode = "select") # set drag event to selection

# generate scatterplot that will update
scatter2 = px.scatter(df, x = "flipper_length_mm", y = "body_mass_g", color = "species")


# set up app layout 
app.layout = html.Div([
    
    html.Div(
        dcc.Graph(id = "g1",
                  figure = scatter1)
    ),
    html.Div(
        dcc.Graph(id = 'g2',
                  figure = scatter2)
    )

])

# The dcc.Graph component has four attributes that can change through user-interaction: 
# hoverData, clickData, selectedData, relayoutData. 
# These properties update when you hover over points, click on points, or select regions of points in a graph.

# callbacks 
@callback(
    Output(component_id='g2', component_property='figure'),
    Input(component_id='g1', component_property='selectedData')
)
def display_selected_data(selectedData):
    if selectedData:
        range = selectedData["range"]
       
        # get end points of selected range 
        x1 = range['x'][0]
        y1 = range['y'][0]
        x2 = range['x'][1]
        y2 = range['y'][1]

        # subset data within ranges
        df_filtered = df[(df['bill_length_mm'] >= x1) & (df['bill_length_mm'] <= x2) &
                         (df['bill_depth_mm'] >= y1) & (df['bill_depth_mm'] <= y2)]
        scatter2_filtered = px.scatter(df_filtered, x = "flipper_length_mm", y = "body_mass_g", color = "species")
    else:
        scatter2_filtered = px.scatter(df, x = "flipper_length_mm", y = "body_mass_g", color = "species")
    return scatter2_filtered

app.run(jupyter_mode="external") # open app externally 

Dash app running on http://127.0.0.1:8050/
